In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Load the dataset
df = pd.read_csv('C:\\Users\\ESHOP\\Documents\\Kaggle Practice Project\\Datasets\\US_Accidents_March23.csv')

In [2]:
# Initial EDA and Preprocessing
print(df.info())
print(df.describe())
print(df['Severity'].value_counts()) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 46 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   ID                     object 
 1   Source                 object 
 2   Severity               int64  
 3   Start_Time             object 
 4   End_Time               object 
 5   Start_Lat              float64
 6   Start_Lng              float64
 7   End_Lat                float64
 8   End_Lng                float64
 9   Distance(mi)           float64
 10  Description            object 
 11  Street                 object 
 12  City                   object 
 13  County                 object 
 14  State                  object 
 15  Zipcode                object 
 16  Country                object 
 17  Timezone               object 
 18  Airport_Code           object 
 19  Weather_Timestamp      object 
 20  Temperature(F)         float64
 21  Wind_Chill(F)          float64
 22  Humidity(%)       

In [3]:
# Handle missing values by dropping columns with a high percentage of missing values
df = df.dropna(thresh=len(df)*0.5, axis=1)

In [ ]:
# Handle remaining missing values with median or mode imputation
df.fillna(df.median(numeric_only=True), inplace=True)
df.fillna(df.mode().iloc[0], inplace=True)

In [ ]:
# Feature Engineering - Date and Time Features
# Convert 'Start_Time' column to datetime with robust parsing options
df['Start_Time'] = pd.to_datetime(df['Start_Time'], format='ISO8601', errors='coerce')

# Verify conversion and check for any NaT values (null values after coercion)
print(df['Start_Time'].isnull().sum())

df['Hour'] = df['Start_Time'].dt.hour
df['DayOfWeek'] = df['Start_Time'].dt.dayofweek
df['Month'] = df['Start_Time'].dt.month

# Drop the original datetime columns to avoid redundancy
df = df.drop(columns=['Start_Time', 'End_Time'], errors='ignore')


0


In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# Step 1: Identify high-cardinality categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns
high_cardinality_cols = [col for col in categorical_cols if df[col].nunique() > 100]

# Step 2: Apply target or ordinal encoding to high-cardinality columns to reduce memory usage
ordinal_encoder = OrdinalEncoder()
df[high_cardinality_cols] = ordinal_encoder.fit_transform(df[high_cardinality_cols])

# Step 3: For remaining low-cardinality columns, apply one-hot encoding
low_cardinality_cols = [col for col in categorical_cols if col not in high_cardinality_cols]
df = pd.get_dummies(df, columns=low_cardinality_cols, drop_first=True)

# Now the dataset should be smaller and more memory-efficient
# Take a sample of the data for testing if the full dataset is too large
df_sample = df.sample(frac=0.1, random_state=42)  # Use 10% of the data


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Define your features and target variable
X = df.drop(columns=['Severity'])  # Replace 'Severity' with your target column
y = df['Severity']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Now apply feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
# Model Training - Random Forest Classifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

In [ ]:
# Model Evaluation
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, model.predict_proba(X_test), multi_class='ovr'))

In [ ]:
# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

In [ ]:
# Feature Importance
feature_importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
plt.figure(figsize=(10, 8))
sns.barplot(x=feature_importances[:20], y=feature_importances.index[:20])  # Top 20 features
plt.title("Top 20 Feature Importances in Predicting Severity")
plt.show()

In [ ]:
# Hyperparameter Tuning with GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10]
}
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)

In [ ]:
# Final Evaluation with Best Model
y_pred_best = best_model.predict(X_test)
print("Tuned Model Accuracy:", accuracy_score(y_test, y_pred_best))
print(classification_report(y_test, y_pred_best))
